In [ ]:
from pybis import Openbis
import json
import ipywidgets as widgets
from IPython.display import display, Javascript
from datetime import datetime

In [ ]:
# Global variables
OPENBIS_SESSION = None

In [ ]:
# Widgets
samples_dropdown = widgets.Dropdown(description='Select sample:', disabled=False)
samples_dropdown.style = {'description_width': 'initial'}
instruments_dropdown = widgets.Dropdown(description='Select instrument:', disabled=False)
instruments_dropdown.style = {'description_width': 'initial'}
sample_details_textbox = widgets.Textarea(value = '', description = '', disabled = True, layout = widgets.Layout(width = '50%', height = '200px'))
create_sputtering_button = widgets.Button(description = 'Save', disabled = False, button_style = '', tooltip = 'Save sputtering information')
quit_button = widgets.Button(description = 'Go back', disabled = False, button_style = '', tooltip = 'Go back')

In [ ]:
# Functions
def read_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Function to close the notebook page
def close_notebook(b):
    display(Javascript(f'window.open("home.ipynb", "_blank")'))
    display(Javascript('window.close()'))

def create_sputtering_action(b):
    global OPENBIS_SESSION
    sample_parents = [samples_dropdown.value, instruments_dropdown.value]
    print(sample_parents)
    # OPENBIS_SESSION.new_object(type = "SPUTTERING", 
    #                         collection = "20240716082516377-60",
    #                         props = {},
    #                         parents = sample_parents).save()

def load_sample_metadata(change):
    global OPENBIS_SESSION
    sample_object = OPENBIS_SESSION.get_object(samples_dropdown.value)
    sample_metadata = sample_object.props.all()
    sample_parents = sample_object.parents
    sample_metadata_string = f" Name: {sample_metadata['$name']}\n"
    for parent in sample_parents:
        parent_object = OPENBIS_SESSION.get_object(parent)
        parent_metadata = parent_object.props.all()
        parent_type = parent_object.type
        sample_metadata_string = f"{sample_metadata_string} {parent_type} {parent_metadata['$name']} is parent."
    sample_details_textbox.value = sample_metadata_string

In [ ]:
# Connect with openBIS
session_data = read_json("token.json")
OPENBIS_SESSION = Openbis(session_data["url"], verify_certificates = False)
OPENBIS_SESSION.set_token(session_data["token"])

## Sputtering

In [ ]:
samples = OPENBIS_SESSION.get_objects(type = "SAMPLE")
instruments = OPENBIS_SESSION.get_objects(type = "INSTRUMENT")

samples_names_permids = [(sample.props["$name"], sample.permId) for sample in samples]
samples_dates = [datetime.fromisoformat(sample.registrationDate) for sample in samples]
print(sorted(samples_dates, 'descend'))
samples_dropdown.options = samples_names_permids
instruments_names_permids = [(instrument.props["$name"], instrument.permId) for instrument in instruments]
instruments_dropdown.options = instruments_names_permids

bottom_buttons_hbox = widgets.HBox([create_sputtering_button, quit_button])
sample_metadata_boxes = widgets.HBox([samples_dropdown, sample_details_textbox])
display(sample_metadata_boxes, instruments_dropdown, bottom_buttons_hbox)
create_sputtering_button.on_click(create_sputtering_action)
quit_button.on_click(close_notebook)
samples_dropdown.observe(load_sample_metadata, names = 'value')